# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [5]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Crouch', 'StandInit', 'Belly', 'HeadBack', 'Left', 'Frog', 'Back', 'Knee', 'Stand', 'Right', 'Sit']


In [16]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [17]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(len(classes)):
    (data,target)=load_pose_data(i)
    all_data += data
    all_target += target

print 'total number of data', len(all_data)

total number of data 222


In [18]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [19]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [22]:
# YOUR CODE HERE
clf.fit(all_data, all_target)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [23]:
clf.predict(all_data[-1]), all_target[-1]

ValueError: Expected 2D array, got 1D array instead:
array=[-0.31289411 -0.23005795 -1.67048407  0.96637797 -0.31289411  0.03532386
 -1.62280262  0.82993603  0.02531069 -0.16758746].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [24]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [25]:
expected = []
predicted = []
# YOUR CODE HERE

predicted = clf.predict(all_data)
expected = all_target

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        30
           1       1.00      1.00      1.00        52
           2       1.00      1.00      1.00        19
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        23
           7       1.00      1.00      1.00        10
           8       1.00      1.00      1.00        11
           9       1.00      0.91      0.95        11
          10       1.00      1.00      1.00        26

   micro avg       1.00      1.00      1.00       222
   macro avg       1.00      0.99      0.99       222
weighted avg       1.00      1.00      1.00       222


Confusion matrix:
[[30  0  0  0  0  0  0  0  0  0  0]
 [ 0 52  0  0  0  0  0  0  0  0  0]
 [ 0  0 19  0  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [26]:
expected = []
predicted = []
# YOUR CODE HERE

predicted = clf.predict(all_data)
expected = all_target

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        30
           1       1.00      1.00      1.00        52
           2       1.00      1.00      1.00        19
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        23
           7       1.00      1.00      1.00        10
           8       1.00      1.00      1.00        11
           9       1.00      0.91      0.95        11
          10       1.00      1.00      1.00        26

   micro avg       1.00      1.00      1.00       222
   macro avg       1.00      0.99      0.99       222
weighted avg       1.00      1.00      1.00       222


Confusion matrix:
[[30  0  0  0  0  0  0  0  0  0  0]
 [ 0 52  0  0  0  0  0  0  0  0  0]
 [ 0  0 19  0  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [27]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [28]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]

ValueError: Expected 2D array, got 1D array instead:
array=[-0.31289411 -0.23005795 -1.67048407  0.96637797 -0.31289411  0.03532386
 -1.62280262  0.82993603  0.02531069 -0.16758746].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.